In [171]:
#Create a tensor-flow session for USE(Universal Setence Encoder)

import tensorflow as tf
import tensorflow_hub as hub


g = tf.Graph()
with g.as_default():
  text_input = tf.placeholder(dtype=tf.string, shape=[None])
  
  
  #Please download USE(Universal Serial Encoder) from below url and unzip and place it locally
  #https://tfhub.dev/google/universal-sentence-encoder-large/3?tf-hub-format=compressed

  embed = hub.Module("C:\\Users\\username\\Downloads\\3")#Enter your location
    
    
  embedded_text = embed(text_input)
  init_op = tf.group([tf.global_variables_initializer(), tf.tables_initializer()])
g.finalize()

session = tf.Session(graph=g)
session.run(init_op)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [212]:
import pandas as pd

#class_data.csv can be found in my GITHUB
df=pd.read_csv("class_data.csv")

df['applicant_id']=df['applicant_id'].astype(str)

In [213]:
columns=list(df.columns)
columns

['applicant_id',
 'first_name',
 'last_name',
 'date_of_birth',
 'country_code',
 'country_name',
 'department',
 'favourite_subject',
 'class_head']

In [214]:
#Replace _ with space for later comparision
dictionary=[]
for d in columns:
        dictionary.append(d.replace("_"," "))
dictionary

['applicant id',
 'first name',
 'last name',
 'date of birth',
 'country code',
 'country name',
 'department',
 'favourite subject',
 'class head']

In [215]:
#Generate all n-grams, here we create uni,bi,tri grams only
from nltk import ngrams

#text=input("Enter your query in plain English?")
text="what is the name and department of applicant 10003?"

text=text.replace("?"," ")
text=text.replace(","," ")
text=text.replace("."," ")

unigram=list(ngrams(text.lower().split(),1))
bigram=list(ngrams(text.lower().split(),2))
trigram=list(ngrams(text.lower().split(),3))
fourgram=list(ngrams(text.lower().split(),4))


In [216]:
#append all generated grams to a list
all_grams=[]
for line in trigram[0:100]:
    new_line=""
    for word in line:
        new_line=new_line+word+" "
    new_line=new_line.rstrip(" ")
    all_grams.append(new_line)
for line in bigram[0:100]:
    new_line=""
    for word in line:
        new_line=new_line+word+" "
    new_line=new_line.rstrip(" ")
    all_grams.append(new_line)
for line in unigram[0:100]:
    new_line=""
    for word in line:
        new_line=new_line+word+" "
    new_line=new_line.rstrip(" ")
    all_grams.append(new_line)
    
all_grams

['what is the',
 'is the name',
 'the name and',
 'name and department',
 'and department of',
 'department of applicant',
 'of applicant 10003',
 'what is',
 'is the',
 'the name',
 'name and',
 'and department',
 'department of',
 'of applicant',
 'applicant 10003',
 'what',
 'is',
 'the',
 'name',
 'and',
 'department',
 'of',
 'applicant',
 '10003']

In [217]:
#Encode all grams to USE(Universal Sentence Encoder) vector
#Encode all column names to USE(Universal Sentence Encoder) vector
query_v = session.run(embedded_text, feed_dict={text_input: all_grams})
dict_v=session.run(embedded_text, feed_dict={text_input: dictionary})
print(query_v.shape)
print(dict_v.shape)

(24, 512)
(9, 512)


In [218]:
#Generate cosine similarit matrix between above two vectors
from sklearn.metrics.pairwise import cosine_similarity

result=[]
r=[]

for i,ii in enumerate(query_v):
    r=[]
    weight=all_grams[i].count(" ")
    for j,jj in enumerate(dict_v):
        similarity=cosine_similarity(query_v[i].reshape(1,512), dict_v[j].reshape(1,512))[0][0]
        r.append(similarity+0.01*weight*similarity)
    result.append(r)
    
print(result)

[[0.18597080647945405, 0.2527309137582779, 0.21155848145484923, 0.16174248576164246, 0.2525496330857277, 0.40213613033294676, 0.2727108496427536, 0.29220599234104155, 0.3084959614276886], [0.2740073096752167, 0.6109123706817627, 0.6153407442569733, 0.29992903232574464, 0.3096180260181427, 0.5244617807865143, 0.3501835548877716, 0.2054035869240761, 0.27221714973449707], [0.4077827489376068, 0.7252013611793519, 0.6719449388980866, 0.34115159153938296, 0.3990750449895859, 0.5641072726249695, 0.37919980108737944, 0.23688327133655548, 0.4247922170162201], [0.5035235971212387, 0.5184328722953796, 0.5343469047546386, 0.3410661721229553, 0.4172597140073776, 0.5247188901901245, 0.7480796599388122, 0.37803934335708617, 0.2644855070114136], [0.383610999584198, 0.400233678817749, 0.39072604179382325, 0.197769872546196, 0.29475352764129636, 0.36940900325775144, 0.7636963963508606, 0.43474850356578826, 0.30067142128944396], [0.7659382152557374, 0.4354214322566986, 0.46668302357196806, 0.396395246386

In [219]:
#Order the similarity matrix in descending
import bisect 
ordered_arr=[]
ordered_index=[]

for i,ii in enumerate(result):
    for j,jj in enumerate(ii):
        #matched greater than 65% is filtered out, you may change this as per your data
        if(jj>=0.65):
            bisect.insort(ordered_arr, jj)
            loc=ordered_arr.index(jj)
            ordered_index.insert(loc,[i,j,jj])


ordered_arr.sort(reverse=True)
ordered_index.reverse()


text_remain=text
list_dum=[]
list_map=[]

#remove identified n-grams from actual text and put in text_remain
#this will help easily identify value in the query like (India,10001,prathul etc)
for i,j in enumerate(ordered_index):
   if(dictionary[ordered_index[i][1]] not in list_dum ):
    print(all_grams[ordered_index[i][0]]+" ==> "+dictionary[ordered_index[i][1]]+" ==> "+str(ordered_index[i][2]))
    list_map.append([all_grams[ordered_index[i][0]],dictionary[ordered_index[i][1]].replace(" ","_"),str(ordered_index[i][2])])
    list_dum.append(dictionary[ordered_index[i][1]])
    text_remain=text_remain.replace(all_grams[ordered_index[i][0]],"")
    
print((list_dum)) 
print("Text without identified n-grams:  "+text_remain)
col_list=[]
for e in list_dum:
    e=e.replace(" ","_")
    col_list.append(e)

col_list


department ==> department ==> 1.0
of applicant ==> applicant id ==> 0.843899490237236
the name and ==> first name ==> 0.7252013611793519
name and ==> last name ==> 0.6815127611160279
['department', 'applicant id', 'first name', 'last name']
Text without identified n-grams:  what is    10003 


['department', 'applicant_id', 'first_name', 'last_name']

In [220]:
#Remove stop words(the,is,of etc)
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS

nlp = English()

my_doc = nlp(text_remain)

token_list = []
for token in my_doc:
    token_list.append(token.text)

filtered_sentence =[] 

for word in token_list:
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        if word!="":
            filtered_sentence.append(word)

filtered_sentence

['   ', '10003']

In [221]:
#find the values in the filtered sentence is matching with any features
q_str=""
qry_col=col_list.copy()
q=""
answer=""
df_ans=df
for e in filtered_sentence:
    for col in col_list:
            if(len(df_ans.loc[(df_ans[col]==e)].values)>0):
                
                df_ans=df_ans.loc[df_ans[col]==e]
                if answer=="":
                    answer="For "+col+" "+e
                else :
                    answer=answer +" and "+col+" "+e
                for i,temp in enumerate(list_map):
                    if(list_map[i][1]==col):
                        for j,temp1 in enumerate(list_map):
                            if(list_map[i][0]==list_map[j][0]):
                                qry_col.remove(list_map[j][1])
df_ans=df_ans[qry_col]
print(answer)
print(df_ans)
df_ans.dtypes

For applicant_id 10003
  department first_name last_name
2        CSE    prathul      nair


department    object
first_name    object
last_name     object
dtype: object

In [222]:
#Important: This function is all the above code wrapped inside
#It is the same code
#This is done so we can call this function and expose via api as done in next cell
def nlp_query(txt):
    from nltk import ngrams
    text=txt

    text=text.replace("?"," ")
    text=text.replace(","," ")
    text=text.replace("."," ")

    unigram=list(ngrams(text.lower().split(),1))
    bigram=list(ngrams(text.lower().split(),2))
    trigram=list(ngrams(text.lower().split(),3))
    fourgram=list(ngrams(text.lower().split(),4))
    
    all_grams=[]
    for line in trigram[0:100]:
        new_line=""
        for word in line:
            new_line=new_line+word+" "
        new_line=new_line.rstrip(" ")
        all_grams.append(new_line)
    for line in bigram[0:100]:
        new_line=""
        for word in line:
            new_line=new_line+word+" "
        new_line=new_line.rstrip(" ")
        all_grams.append(new_line)
    for line in unigram[0:100]:
        new_line=""
        for word in line:
            new_line=new_line+word+" "
        new_line=new_line.rstrip(" ")
        all_grams.append(new_line)
        
    print(all_grams)
    
    query_v = session.run(embedded_text, feed_dict={text_input: all_grams})
    dict_v=session.run(embedded_text, feed_dict={text_input: dictionary})
    print(query_v.shape)
    print(dict_v.shape)
    
    from sklearn.metrics.pairwise import cosine_similarity

    result=[]
    r=[]

    for i,ii in enumerate(query_v):
        r=[]
        weight=all_grams[i].count(" ")
        for j,jj in enumerate(dict_v):
            similarity=cosine_similarity(query_v[i].reshape(1,512), dict_v[j].reshape(1,512))[0][0]
            r.append(similarity+0.01*weight*similarity)
        result.append(r)

    print(result)
    
    import bisect 
    ordered_arr=[]
    ordered_index=[]

    for i,ii in enumerate(result):
        for j,jj in enumerate(ii):
            if(jj>=0.65):
                bisect.insort(ordered_arr, jj)
                loc=ordered_arr.index(jj)
                ordered_index.insert(loc,[i,j,jj])


    ordered_arr.sort(reverse=True)
    ordered_index.reverse()


    text_remain=text
    list_dum=[]
    list_map=[]
    for i,j in enumerate(ordered_index):
       if(dictionary[ordered_index[i][1]] not in list_dum ):
        print(all_grams[ordered_index[i][0]]+" ==> "+dictionary[ordered_index[i][1]]+" ==> "+str(ordered_index[i][2]))
        list_map.append([all_grams[ordered_index[i][0]],dictionary[ordered_index[i][1]].replace(" ","_"),str(ordered_index[i][2])])
        list_dum.append(dictionary[ordered_index[i][1]])
        text_remain=text_remain.replace(all_grams[ordered_index[i][0]],"")

    print((list_dum)) 
    print("Text without identified n-grams:  "+text_remain)
    col_list=[]
    for e in list_dum:
        e=e.replace(" ","_")
        col_list.append(e)

    col_list

    from spacy.lang.en import English
    from spacy.lang.en.stop_words import STOP_WORDS

    nlp = English()

    my_doc = nlp(text_remain)

    token_list = []
    for token in my_doc:
        token_list.append(token.text)

    filtered_sentence =[] 

    for word in token_list:
        lexeme = nlp.vocab[word]
        if lexeme.is_stop == False:
            if word!="":
                filtered_sentence.append(word)

    filtered_sentence
    q_str=""
    qry_col=col_list.copy()
    q=""
    answer=""
    df_ans=df
    for e in filtered_sentence:
        for col in col_list:
                if(len(df_ans.loc[(df_ans[col]==e)].values)>0):

                    df_ans=df_ans.loc[df_ans[col]==e]
                    if answer=="":
                        answer="For "+col+" "+e
                    else :
                        answer=answer +" and "+col+" "+e
                    for i,temp in enumerate(list_map):
                        if(list_map[i][1]==col):
                            for j,temp1 in enumerate(list_map):
                                if(list_map[i][0]==list_map[j][0]):
                                    qry_col.remove(list_map[j][1])
    df_ans=df_ans[qry_col]
    print(answer)
    print(df_ans)
    
    html_txt= "<br>"+"User Question:<br>&nbsp <b> " + txt+"</b><br><br>Computer response:<br>&nbsp<b>"+answer+"</b>  we have <br>"
    html_txt=html_txt+"<br>"+df_ans.to_html()
    html_txt=html_txt+'<a href="http://127.0.0.1:5000/"> \
    <button>Go Back</button>\
    </a> <br><br><br>---------------- by himeshph    '
    return html_txt



In [ ]:
#Create a flask app to call the above function
#By default the flask app run on http://127.0.0.1:5000/
#Once cell is run go to this url http://127.0.0.1:5000/

from flask import Flask,request
app = Flask(__name__)

@app.route("/")
def hello():
    html="<b><center><font size='50'>Welcome to magic query</font></center></b><br><br>"
    html=html+df.to_html()
    html=html+'<br><form method="POST" action="http://127.0.0.1:5000/submit">\
    Please enter your query here: <input type="text" size="45" name="txt" id="txt"/>\
    <br><br><input type="submit" name="submit"/> \
    <br><br>Sample questions:\
    <br> 1. what is the name of applicant 10002?\
    <br> 2. hey computer, tell me favorite subject of name anand?\
    <br> 3. list me names of applicant in department AID?\
    <br> 4. tell me the birth date, country, and department of name anjali? \
    <br> 5. which applicant born on 04-JAN-1993 and what is his name? \
    <br> 6. applicants from country USA and their names and department?\
    <br><br><br>---------------- by himeshph   \
    </form>'
    return html

@app.route("/submit",methods=['POST'])
def submit():
    txt=request.form['txt']
    html="<b><center><font size='50'>Welcome to magic query</font></center></b><br><br>"
    html=html+df.to_html()
    html_txt=nlp_query(txt)
    html=html+html_txt
    return html

if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: development
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [03/Dec/2019 13:18:49] "GET / HTTP/1.1" 200 -


['what is the', 'is the name', 'the name of', 'name of applicant', 'of applicant 10002', 'what is', 'is the', 'the name', 'name of', 'of applicant', 'applicant 10002', 'what', 'is', 'the', 'name', 'of', 'applicant', '10002']
(18, 512)
(9, 512)
[[0.18597080647945405, 0.2527309137582779, 0.21155848145484923, 0.16174248576164246, 0.2525496330857277, 0.40213613033294676, 0.2727108496427536, 0.29220599234104155, 0.3084959614276886], [0.2740073096752167, 0.6109123706817627, 0.6153407442569733, 0.29992903232574464, 0.3096180260181427, 0.5244617807865143, 0.3501835548877716, 0.2054035869240761, 0.27221714973449707], [0.4062927424907684, 0.7222715663909912, 0.6838851141929626, 0.3845921981334686, 0.46318262100219726, 0.6128269207477569, 0.46507121086120606, 0.3036492758989334, 0.47187071800231933], [0.8291172122955323, 0.6463967514038086, 0.6691997230052948, 0.46535865783691405, 0.4728176879882813, 0.48009986042976377, 0.5259053993225098, 0.297783819437027, 0.39035244584083556], [0.768296824693

127.0.0.1 - - [03/Dec/2019 13:19:04] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2019 13:19:22] "GET / HTTP/1.1" 200 -


['which applicant born', 'applicant born on', 'born on 04-jan-1993', 'on 04-jan-1993 and', '04-jan-1993 and what', 'and what is', 'what is his', 'is his name', 'which applicant', 'applicant born', 'born on', 'on 04-jan-1993', '04-jan-1993 and', 'and what', 'what is', 'is his', 'his name', 'which', 'applicant', 'born', 'on', '04-jan-1993', 'and', 'what', 'is', 'his', 'name']
(27, 512)
(9, 512)
[[0.7085240256786346, 0.43819023728370665, 0.4609634184837341, 0.4845591187477112, 0.3371748769283295, 0.4191902232170105, 0.2935005068778992, 0.29032056391239164, 0.13521308451890945], [0.7909287142753602, 0.4826393401622772, 0.5124936997890472, 0.6579589533805847, 0.46951703310012816, 0.41183910727500916, 0.3584689950942993, 0.24009853720664978, 0.3110657787322998], [0.3546977126598358, 0.44052936136722565, 0.4363837230205536, 0.5751469874382019, 0.2779542946815491, 0.33157501101493836, 0.13831096708774568, 0.22964753240346908, 0.31066847205162046], [0.26333590149879454, 0.23632337898015976, 0.2

127.0.0.1 - - [03/Dec/2019 13:19:36] "POST /submit HTTP/1.1" 200 -
127.0.0.1 - - [03/Dec/2019 13:19:52] "GET / HTTP/1.1" 200 -
